In [11]:
#Fav / Dog Win percentage + odds filter
import pandas as pd
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import datetime
stake=100
datefilter='2024-01-01'
datefilteryesterday='2025-02-13'

devengine = create_engine("sqlite:///C:/Git/tennis_atp/database/bets_sqllite.db")
time_now = datetime.datetime.now()

def hist_data_dog(surface, sex, odds_low,odds_high,date):
    data=pd.read_sql_query(f"Select distinct * from results_{surface}_1 where date > '{date}' and date < '{datefilteryesterday}'",con=devengine)#.to_clipboard(index=False)
    data["Fav_Odds"] = data.Fav_Odds.astype(float)
    data["Dog_Odds"] = data.Dog_Odds.astype(float)
    data['Fav_Win'] = data['Winner']==data['Fav']
    data['DogWin'] = data['Winner']!=data['Fav']
    data=data[(data['Fav_Odds']>=odds_low)&(data['Fav_Odds']<=odds_high)]
    data=data[data['Sex']==sex]
    return data
hard_womens=hist_data_dog('HArd','Womens',1,2,datefilter)
hard_mens=hist_data_dog('HArd','Mens',1,2,datefilter)
clay_womens=hist_data_dog('Clay','Womens',1,2,datefilter)
clay_mens=hist_data_dog('Clay','Mens',1,2,datefilter)
temp=pd.concat([hard_womens,hard_mens]).sort_values(by='Date')
temp = temp.reset_index(drop=True)
temp['Fav_Loss'] = (temp['Fav_Win'] == 0).astype(int)
temp['Fav_Loss_Count'] = temp.groupby('Fav')['Fav_Loss'].cumsum()
temp['Dog_Win'] = (temp['Fav_Win'] == 0).astype(int)
temp['Dog_Win_Count'] = temp.groupby('Dog')['Dog_Win'].cumsum()
# Load the dataset
df = temp[(temp['Sex']!='Wobmens')&(temp['Fav_Odds']<2)&(temp['Fav_Odds']>1.4)]


# Feature selection and preprocessing
features = ['fav_percent', 'dog_percent', 'Elo_Fav_Elo', 'Elo_Dog_Elo', 
            'fav_rank', 'dog_rank', 'dog_rank_high', 'fav_rank_high','Fav_Top100','Dog_Top100','Dog_Win_Count','Fav_Loss_Count']
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Drop the 'Winner_Odds' column
df = df.drop(columns=['Winner_Odds'])


target = 'Fav_Win'

X = df[features]  # Features for training
y = df[target]  # Target variable

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)


# Split based on time (e.g., last 20% for testing)
split_index = int(len(X) * 0.95)
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

# Model training
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Prediction and evaluation
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Add predictions and actuals to the test set DataFrame
test_with_predictions = X_test.copy()
test_with_predictions['Actual'] = y_test
test_with_predictions['Predicted'] = y_pred

# Add Fav_Odds and Dog_Odds back to the test set for profit calculation
test_with_predictions = test_with_predictions.merge(
    df[['Date','Fav','Dog','Fav_Odds', 'Dog_Odds']], left_index=True, right_index=True
)

# Define the stake amount
stake = 100  # Example stake amount

# Profit calculation
def calculate_profit(row):
    if row['Predicted'] == 1:  # If the model predicts the favorite wins
        if row['Actual'] == 1:  # If the prediction is correct
            return stake * (row['Fav_Odds'] - 1)  # Profit based on favorite odds
        else:
            return -stake  # Loss of the stake
    else:  # If the model predicts the underdog wins
        if row['Actual'] == 0:  # If the prediction is correct
            return stake * (row['Dog_Odds'] - 1)  # Profit based on underdog odds
        else:
            return -stake  # Loss of the stake

test_with_predictions['Profit'] = test_with_predictions.apply(calculate_profit, axis=1)

# Save the output DataFrame to clipboard
fav_predictions=test_with_predictions[test_with_predictions['Predicted']==1].copy()
# Calculate overall profit
fav_profit = fav_predictions['Profit'].sum()
#print(f"Fav Profit: ${fav_profit:.2f}",len(fav_predictions))

dog_predictions=test_with_predictions[test_with_predictions['Predicted']==0].copy()
# Calculate overall profit
dog_profit = dog_predictions['Profit'].sum()
#print(f"Dog Profit: ${dog_profit:.2f}", len(dog_predictions))
dog_predictions.to_clipboard()
import numpy as np

# Define the odds bands (10c increments from 1.00 to 2.00)
bins = np.arange(1.0, 2.1, 0.1)  # Create bins: [1.0, 1.1, ..., 2.0]
labels = [f"{bins[i]:.1f} to {bins[i+1]:.1f}" for i in range(len(bins) - 1)]

# Create a new column for odds bands
dog_predictions['Fav_Odds_Band'] = pd.cut(dog_predictions['Fav_Odds'], bins=bins, labels=labels, right=False)

# Add a column to indicate if the prediction was correct
dog_predictions['Correct_Prediction'] = dog_predictions['Actual'] == dog_predictions['Predicted']

# Group by Fav_Odds_Band and calculate Profit, count of records, and winning percentage
grouped_dog = dog_predictions.groupby('Fav_Odds_Band').agg(
    Total_Profit=('Profit', 'sum'),
    Record_Count=('Profit', 'count'),
    Winning_Percentage=('Correct_Prediction', lambda x: (x.sum() / len(x) * 100) if len(x) > 0 else 0)
).reset_index()

# Print the results
#print(grouped_dog)



import numpy as np

# Define the odds bands (10c increments from 1.00 to 2.00)
bins = np.arange(1.0, 2.1, 0.1)  # Create bins: [1.0, 1.1, ..., 2.0]
labels = [f"{bins[i]:.1f} to {bins[i+1]:.1f}" for i in range(len(bins) - 1)]

# Create a new column for odds bands
fav_predictions['Fav_Odds_Band'] = pd.cut(fav_predictions['Fav_Odds'], bins=bins, labels=labels, right=False)

# Add a column to indicate if the prediction was correct
fav_predictions['Correct_Prediction'] = fav_predictions['Actual'] == fav_predictions['Predicted']

# Group by Fav_Odds_Band and calculate Profit, count of records, and winning percentage
grouped_fav = fav_predictions.groupby('Fav_Odds_Band').agg(
    Total_Profit=('Profit', 'sum'),
    Record_Count=('Profit', 'count'),
    Winning_Percentage=('Correct_Prediction', lambda x: (x.sum() / len(x) * 100) if len(x) > 0 else 0)
).reset_index()

# Print the results
#print(grouped_fav)


Accuracy: 0.6108949416342413
              precision    recall  f1-score   support

       False       0.56      0.74      0.64       118
        True       0.69      0.50      0.58       139

    accuracy                           0.61       257
   macro avg       0.63      0.62      0.61       257
weighted avg       0.63      0.61      0.61       257



In [12]:
features

['fav_percent',
 'dog_percent',
 'Elo_Fav_Elo',
 'Elo_Dog_Elo',
 'fav_rank',
 'dog_rank',
 'dog_rank_high',
 'fav_rank_high',
 'Fav_Top100',
 'Dog_Top100',
 'Dog_Win_Count',
 'Fav_Loss_Count']

In [13]:
#Fav / Dog Win percentage + odds filter
import pandas as pd
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import datetime
stake=100
datefilter='2024-01-01'
datefilteryesterday='2025-02-14'

devengine = create_engine("sqlite:///C:/Git/tennis_atp/database/bets_sqllite.db")
time_now = datetime.datetime.now()

def hist_data_dog(surface, sex, odds_low,odds_high,date):
    data=pd.read_sql_query(f"Select distinct * from results_{surface}_1 where date > '{date}' and date < '{datefilteryesterday}'",con=devengine)#.to_clipboard(index=False)
    data["Fav_Odds"] = data.Fav_Odds.astype(float)
    data["Dog_Odds"] = data.Dog_Odds.astype(float)
    data['Fav_Win'] = data['Winner']==data['Fav']
    data['DogWin'] = data['Winner']!=data['Fav']
    data=data[(data['Fav_Odds']>=odds_low)&(data['Fav_Odds']<=odds_high)]
    data=data[data['Sex']==sex]
    return data

hard_womens=hist_data_dog('HArd','Womens',1,2,datefilter)
hard_mens=hist_data_dog('HArd','Mens',1,2,datefilter)
clay_womens=hist_data_dog('Clay','Womens',1,2,datefilter)
clay_mens=hist_data_dog('Clay','Mens',1,2,datefilter)
temp=pd.concat([hard_womens,hard_mens]).sort_values(by='Date')
temp = temp.reset_index(drop=True)
temp['Fav_Loss'] = (temp['Fav_Win'] == 0).astype(int)
temp['Fav_Loss_Count'] = temp.groupby('Fav')['Fav_Loss'].cumsum()
temp['Dog_Win'] = (temp['Fav_Win'] == 0).astype(int)
temp['Dog_Win_Count'] = temp.groupby('Dog')['Dog_Win'].cumsum()
cunt=temp

# Filter matches after 2024-12-31
today = temp[temp['Date'] > '2025-02-12'].copy()
today.loc[today['Fav_Win'] == False, 'Fav_Loss_Count'] -= 1
today.loc[today['Fav_Win'] == False, 'Dog_Win_Count'] -= 1
today["Fav_Odds"] = today.Fav_Odds.astype(float)
today["Dog_Odds"] = today.Dog_Odds.astype(float)
# Apply odds filter
today = today[today['Fav_Odds'] > 1.4]
print(cunt)
# Prepare features and make predictions
today_features = today[features]
today['Predicted'] = model.predict(today_features)
# Predict probabilities for today's features
probabilities = model.predict_proba(today_features)

# Extract the probability of the positive class (e.g., `Fav_Win` = 1)
today['Predicted_Prob'] = probabilities[:, 0]  # Column 1 corresponds to the positive class

today['Time']='0'
dog=today[today['Predicted']==False][['Date','Time','Dog','Predicted','Predicted_Prob','Dog_Odds','Fav_Odds','Fav']]
dog = dog[dog['Dog_Odds'] >1]
dog = dog[dog['Predicted_Prob']>0.65]

fav=today[today['Predicted']==True][['Date','Time','Fav','Predicted','Predicted_Prob','Dog_Odds','Fav_Odds','Dog']]
fav = fav[fav['Dog_Odds'] >1]
fav = fav[fav['Predicted_Prob']<0.1]



# Add a column to calculate profit/loss with a stake of 100 units
# Profit = (Dog_Odds - 1) * stake if the dog wins, otherwise -stake
stake = 100
today= today[today['Dog_Odds'] >1]
today = today[today['Predicted_Prob']>0.65]
today['Profit'] = today.apply(
    lambda row: (row['Dog_Odds'] - 1) * stake if row['Winner'] == row['Dog'] else -stake, axis=1
)

# Calculate total profit
total_profit = today['Profit'].sum()

# Print the total profit
print("Total Profit:", total_profit, len(today),len(today[today['Dog']==today['Winner']])/len(today))

# Save the data to clipboard
today.to_clipboard()

            Date Winner_Odds                      Winner  Fav_Odds  Dog_Odds  \
0     2024-01-02        1.66                Petra Martic      1.66      2.21   
1     2024-01-02        1.03               Nicolas Jarry      1.03     15.00   
2     2024-01-02        1.55           Miomir Kecmanovic      1.55      2.42   
3     2024-01-02        2.69             Sebastian Ofner      1.46      2.69   
4     2024-01-02        1.63  Giovanni Mpetshi Perricard      1.63      2.15   
...          ...         ...                         ...       ...       ...   
8142  2025-02-13        1.64             Francesca Jones      1.64      2.19   
8143  2025-02-13        2.94          Selena Janicijevic      1.37      2.94   
8144  2025-02-13        2.15              Katarzyna Kawa      1.67      2.15   
8145  2025-02-13        2.06               Filip Misolic      1.70      2.06   
8146  2025-02-13        1.48             Dominik Koepfer      1.48      2.54   

                             Fav       

In [14]:
#temp.sort_values(by='Fav_Loss_Count',ascending=False)
#temp.sort_values(by='Dog_Win_Count',ascending=False)

In [15]:
dog#.to_sql('DailyPredictions_Dog',con=devengine,index=False,if_exists='append')

,Date,Time,Dog,Predicted,Predicted_Prob,Dog_Odds,Fav_Odds,Fav
8144,2025-02-13,0,Katarzyna Kawa,False,0.82,2.15,1.67,Louisa Chirico


In [16]:
fav#.to_sql('DailyPredictions_Fav',con=devengine,index=False,if_exists='append')

,Date,Time,Fav,Predicted,Predicted_Prob,Dog_Odds,Fav_Odds,Dog
8132,2025-02-13,0,Daniel Rincon,True,0.01,2.26,1.59,Pedro Cachin
8142,2025-02-13,0,Francesca Jones,True,0.04,2.19,1.64,Sachia Vickery
